# Data Visualisation of Workflow Outputs

In [ ]:
pip install requests

### Set required variables

In [2]:
# Set the location of the workflow output
WORKFLOW_OUTPUT_ASSET = ''

# Set your workspace API key
WORKSPACE_API_KEY = ''

# Environment
environment = 'staging'

### Previewing the Asset before serving it

Experiment with different rescales and colormap options to see what works best for your data.

Example colormaps: `'accent', 'accent_r', 'afmhot', 'afmhot_r', 'algae', 'algae_r', 'amp', 'amp_r', 'autumn', 'autumn_r', 'balance', 'balance_r', 'binary', 'binary_r', 'blues', 'blues_r', 'bone', 'bone_r', 'brbg', 'brbg_r', 'brg', 'brg_r', 'bugn', ...`

In [ ]:
import requests
from IPython.display import Image, display

TITILER_PREVIEW_URL = f'https://{environment}.eodatahub.org.uk/titiler/core/cog/preview'
TITILER_PREVIEW_PARAMS = {
    'url': WORKFLOW_OUTPUT_ASSET,
    'bidx': 1,
    'rescale': '9,255',
    'colormap_name': 'rain_r'
}

response = requests.get(TITILER_PREVIEW_URL, params=TITILER_PREVIEW_PARAMS, headers={'Authorization': f'Bearer {WORKSPACE_API_KEY}'})

# Display the image
image = Image(response.content)
display(image)

### Visualise this in a 3rd Party Application (QGIS)

To visualise the data in a 3rd party application, we can use the Hub's OGC XYZ Tile Service.

This OGC service works both for public and private datasets. For private datasets, you will need to add an `Authorization` header to your requests.

#### Public

This is an example public ESA CCI Land Cover dataset we can use as a basemap.

```
https://staging.eodatahub.org.uk/titiler/core/cog/tiles/WebMercatorQuad/{z}/{x}/{y}?scale=1&url=https%3A%2F%2Fdap.ceda.ac.uk%2Fneodc%2Fesacci%2Fland_cover%2Fdata%2Fland_cover_maps%2Fv2.0.7%2FESACCI-LC-L4-LCCS-Map-300m-P1Y-2015-v2.0.7.tif&bidx=1&rescale=0%2C300&colormap_name=rainbow
```

In [37]:
TITILER_TILES_URL = 'https://' + environment + '.eodatahub.org.uk/titiler/core/cog/tiles/WebMercatorQuad/{z}/{x}/{y}'

TITILER_TILES_URL + '?' + '&'.join([f'{k}={v}' for k, v in TITILER_PREVIEW_PARAMS.items()])


'https://staging.eodatahub.org.uk/titiler/core/cog/tiles/WebMercatorQuad/{z}/{x}/{y}?url=https://workspaces-eodhp-staging.s3.eu-west-2.amazonaws.com/james-hinton/outputs/S2A_20230213_lat57lon209_T30VWJ_ORB037_utm30n_osgb_vmsk_sharp_rad_srefdem_stdsref.tif&bidx=1&rescale=9,255&colormap_name=rain_r'

## Publish Workspace File

We can publish workspace results by copying them to the public directory. This example does it for an S3 file, but it also applies to an EFS file.

In [ ]:
import boto3
import requests

env = "staging"
workspace = "sgillies-tpzuk"
workspaces_bucket = f"workspaces-eodhp-{env}"
filepath = "commercial-data/airbus/cat_68c72942-eddf-11ef-b4a3-ca3e8cd28499.json"

# Confirm file is not published
resp = requests.head(
    f"https://{workspace}.{env}.eodatahub-workspaces.org.uk/files/workspaces-eodhp-{env}/{filepath}", 
    headers={"accept": "application/json"}
)
print(f"File is not publically accessible: status={resp.status_code}")

# Publish the file
s3 = boto3.client("s3")
src = f"{workspace}/{filepath}"
dest = f"{workspace}/public/{filepath}"
print(f"Copying {src} to {dest} in bucket {workspaces_bucket}...")
s3.copy(
    {
        "Bucket": workspaces_bucket, 
        "Key": src
    }, 
    workspaces_bucket, 
    dest,
)

# Confirm file is published
resp = requests.head(
    f"https://{workspace}.{env}.eodatahub-workspaces.org.uk/files/workspaces-eodhp-{env}/public/{filepath}", 
    headers={"accept": "application/json"}
)
print(f"File is publically accessible: status={resp.status_code}")


File is not publically accessible: status=401


ClientError: An error occurred (403) when calling the HeadObject operation: Forbidden